Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
import sqlite3
import pandas as pd

# File paths for your CSV files
csv_files = [
    'D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/foursquare_data.csv',
    'D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/yelp_data.csv',
    'D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/montreal_df.csv'
]

# Function to create SQLite database and tables
def create_db_and_tables():
    conn = sqlite3.connect('Montreal_POI_BIKE.db')  # Connecting to SQLite database
    cursor = conn.cursor()

    # Create tables for each CSV file
    for csv_file in csv_files:
        table_name = csv_file.split('/')[-1].split('.')[0]  # Extract table name from file path
        df = pd.read_csv(csv_file)  # Reading CSV into DataFrame

        # Convert DataFrame to SQL table
        df.to_sql(table_name, conn, if_exists='replace', index=False)

#Confirm that the Table was created 
        print(f"Table '{table_name}' created successfully.")

    conn.close()

# in researching for method to combine these dataframes found this would limit the execution of the code to prevent it from 
#being run automatically. This may cause data to be overwritten which could erase steps to clean data. 
if __name__ == "__main__":
    create_db_and_tables()

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
# Create a heatmap
#This code was run in the EDA notebook
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

NameError: name 'plt' is not defined

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

The code above was used to create the database using a function. Stored CSV files into Montreal_POI_BIKE.db as tables

Look at the data before and after the join to validate your data.

In [ ]:
#example of checking data before joining and altering

conn = sqlite3.connect('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/Montreal_POI_BIKE.db')

#Reading tables into pandas DataFrames
montreal_data = pd.read_sql_query("SELECT * FROM montreal_df;", conn)
foursquare_data = pd.read_sql_query("SELECT * FROM foursquare_data;", conn)
yelp_data = pd.read_sql_query("SELECT * FROM yelp_data;", conn)

# Close the database connection
conn.close()



In [ ]:
#checked how each column fit in table. 
#then changed names to better match each table
montreal_data.head()
foursquare_data.head()
yelp_data.head()

In [ ]:

#change colum names to fit the other tables
montreal_data = montreal_data.rename(columns={
    'name': 'Bike Station Name',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'free_bikes': 'Free_Bikes',
    'empty_slots': 'Empty_Slots'
    })

foursquare_data.rename(columns={
    'Name': 'Venue Name'
      })
yelp_data.rename(columns={
    'Name': 'Venue Name'
      })

In [ ]:
# Merge montreal_data with foursquare_data
merged_df = pd.merge(montreal_data, foursquare_data, on=['Latitude', 'Longitude'], how='left')

# Merge montreal_data with yelp_data
merged_df = pd.merge(merged_df, yelp_data, on=['Latitude', 'Longitude'], how='left')

In [ ]:
merged_df.isna().sum()

In [ ]:
# Drop columns with no values (all null)
columns_to_drop = ['Name_x', 'Address_x', 'Categories', 'Rating_x', 'TotalRatings']
merged_df = merged_df.drop(columns=columns_to_drop)

# Display the remaining columns
print(merged_df.columns)

In [ ]:
# Filter rows where 'Name_y', 'Rating_y', 'Address_y' are not null
merged_df = merged_df[merged_df['Name_y'].notnull() & 
                      merged_df['Rating_y'].notnull() & 
                      merged_df['Address_y'].notnull()]

# Display the remaining rows
print(merged_df.isna().sum())